# Import modules

In [ ]:
import torch

from model import FeNeC

from configs.imagenet_subset_resnet import config
from utils.loader import FeNeCDataLoader

# Choose the device

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: cuda


# Run the training and evalutating loop

For simplicity, you can pass extracted features from backbone with shapes as below to the model:
- X_train (num_train_samples, num_features)
- y_train (num_train_samples)
- X_test (num_test_samples, num_features)
- y_test (num_test_samples)

Here we are using our dataloader, but it requires the datasets to be stored in
the given data directory and the directory with dataset name (ex. "./data/tinyimagenet") Each task data need to be stored
in task_0.hdf5 files were 0 is number of task. In each of hdf5 files there are some keys specified. 
Look at utils/loader.py file for more information.

In [ ]:
import torch
import numpy as np
from collections import defaultdict

num_tasks = 6
num_classes = 100
accuracies = []

data_loader = FeNeCDataLoader(
    num_tasks=num_tasks,
    dataset_name="imagenet_subset_path",
    load_covariances=True,
    load_prototypes=False,
    dataset_path="./data",
)
model = FeNeC(config, device=device)

# Matrix to hold per-task per-class accuracy
per_class_accuracy_matrix = np.zeros((num_tasks, num_classes))

for task_id in range(num_tasks):
    X_train, y_train, X_test, y_test, covariances, prototypes = data_loader.get_data(task_id)

    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    # Overall accuracy
    accuracy = (
        torch.sum((y_test.flatten().to(device) == predictions).int()) / X_test.shape[0]
    ).item()
    accuracies.append(accuracy)
    print(f"Task {task_id} - Accuracy: {accuracy}")

    # Per-class accuracy
    y_test_cpu = y_test.flatten().cpu()
    preds_cpu = predictions.cpu()

    for cls in range(num_classes):
        mask = y_test_cpu == cls
        if mask.sum().item() == 0:
            acc = np.nan  # No samples for this class
        else:
            correct = (preds_cpu[mask] == cls).sum().item()
            acc = correct / mask.sum().item()

        per_class_accuracy_matrix[task_id, cls] = acc

# Print average accuracy
print(f"\nAverage incremental accuracy: {sum(accuracies)/len(accuracies)}")

Task 0 - Accuracy: 0.8121910691261292
Task 1 - Accuracy: 0.7651376128196716
Task 2 - Accuracy: 0.7211764454841614
Task 3 - Accuracy: 0.7169323563575745
Task 4 - Accuracy: 0.6973500847816467
Task 5 - Accuracy: 0.6715887188911438
Task 6 - Accuracy: 0.6632136106491089
Task 7 - Accuracy: 0.6534972786903381
Task 8 - Accuracy: 0.644777774810791
Task 9 - Accuracy: 0.6371357440948486

Average incremental accuracy: 0.6983000695705414


In [17]:
per_class_accuracy_matrix.shape

(10, 200)

In [ ]:
num_classes_task_0 = 50
num_classes_per_task = 10
bwt_list = [np.mean(per_class_accuracy_matrix[-1][:num_classes_task_0]) - np.mean(per_class_accuracy_matrix[0][:num_classes_task_0])]

for i in range(1, num_tasks-1):
    bwt_list.append(np.mean(per_class_accuracy_matrix[-1][i*num_classes_per_task + num_classes_task_0 - num_classes_per_task: i*num_classes_per_task + num_classes_task_0]) - np.mean(per_class_accuracy_matrix[i][i*num_classes_per_task + num_classes_task_0 - num_classes_per_task: i*num_classes_per_task + num_classes_task_0]))

bwt = np.mean(bwt_list) * 100
print(bwt)

-6.7522401866910995


In [ ]:
# data_from_kris = [[0.840],
# [0.8064,0.576],
# [0.793,0.536,0.601],
# [0.780,0.495,0.577,0.536],
# [0.7676,0.481,0.553,0.521,0.511],
# [0.7584,0.453,0.547,0.501,0.502,0.561]]

# bwt = 0
# for i in range(len(data_from_kris)-1):
#     bwt += data_from_kris[-1][i] - data_from_kris[i][-1]
# bwt /= 5

# print(bwt)

In [ ]:

print(f"BWT ResNet CIFAR-100: {-6.051999999999998}")
print(f"BWT ResNet TinyImageNet: {-6.912033153448428}")
print(f"BWT ResNet Imagenet-Subset: {1}")
print(f"BWT ViT CIFAR: {-4.444444444444437}")
print(f"BWT ViT ImageNet-R: {6.7522401866910995}")

BWT ResNet CIFAR-100: -6.051999999999998
BWT ResNet TinyImageNet: -6.912033153448428
BWT ResNet Imagenet-Subset: 1
BWT ViT CIFAR: -4.444444444444437
BWT ViT ImageNet-R: -6.077016168021989
